In [190]:
!pip install pandas

!pip install seaborn
!pip install numpy

!pip install matplotlib

!pip install scikit-learn

You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
import matplotlib.pyplot as plt

import matplotlib

import seaborn as sns

import numpy as np

import pandas as pd

from sklearn import neighbors, metrics

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from collections import Counter

import re

from sklearn.covariance import EllipticEnvelope

from sklearn.linear_model import LinearRegression

from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score

# functions

In [8]:
def get_metrics(targets: np.ndarray, predicted_targets: np.ndarray, mode: str):
    '''

    :param targets:
    :param predicted_targets:
    :param mode: influence to return: mode = "show_metrics" - print metrics to the Console;
                                        mode = "return_metrics" - return dict which consists all metrics values
                                        with their names as keys
    :return:
    {
    "accuracy": accuracy_value, "precision": precision_value,
    "recall": recall_value, "f1_score": f1_score_value,
    "confusion_matrix": confusion_matrix
    }
    '''
    # token = (actual target, predicted target)
    # 1 - positive target, 0 - negative target
    tokens = [(j, i) for i in [1, 0] for j in [1, 0]]
    # tokens = [true_pos_token,
    #           false_pos_token,
    #           false_neg_token,
    #           true_neg_token]
    c = Counter(zip(targets, predicted_targets))
    [true_pos_value,
     false_pos_value,
     false_neg_value,
     true_neg_value] = [c[token] for token in tokens]
    accuracy = (true_neg_value + true_pos_value) / len(targets)
    confusion_matrix = np.array([[true_pos_value, false_pos_value],
                                 [false_neg_value, true_neg_value]])
    precision = true_pos_value / (true_pos_value + false_pos_value)
    recall = true_pos_value / (true_pos_value + false_neg_value)
    f1_score = 2 * precision * recall / (precision + recall)
    if mode == "show_metrics":
        print(f"accuracy = {accuracy}")
        print(f"confusion_matrix = \n{confusion_matrix}")
        print(f"precision = {precision}")
        print(f"recall = {recall}")
        print(f"f1_score = {f1_score}")
    elif mode == "return_metrics":
        res_dict = {"accuracy": accuracy, "confusion_matrix": confusion_matrix}
        return res_dict

In [9]:
preproc_data = pd.read_csv("pre-processing_BankChurners.csv")
x = preproc_data.drop(["Attrition_Flag",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
                       ], axis=1).values
y = preproc_data["Attrition_Flag"].values
preproc_data

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,0,-0.165261,1.060261,0.503636,-0.893897,0.522808,0.425471,-0.263546,0.384495,0.763915,...,0.447062,-0.473285,0.489410,2.666758,-0.960392,-0.975049,3.892696,-0.776043,-0.437752,0.437762
1,0,0.583481,1.060261,0.503636,0.593115,0.522808,1.308928,-0.263546,0.008956,0.120645,...,-0.573420,-1.426735,-0.445377,8.499347,-0.741457,-1.913197,6.910076,-0.997334,-0.437950,0.437953
2,0,-0.789213,-0.943070,1.273662,-0.893897,0.522808,-0.850632,-0.263546,-0.241404,-0.522625,...,-0.584975,1.661853,-0.733868,2.991048,-0.952436,-1.913197,6.910076,1.759732,-0.437642,0.437652
3,0,-0.789213,1.060261,0.503636,-1.637403,0.522808,0.425471,-0.263546,-1.868743,0.763915,...,-0.430577,-1.426735,-0.302563,6.558239,-1.057042,-1.572052,7.621803,-0.997334,-0.437949,0.437953
4,0,-0.290051,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,-0.522625,...,-0.508271,0.103448,-0.517445,2.856699,-0.976893,-1.742625,0.572725,0.130886,-0.437857,0.437844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,0,0.458691,1.060261,-0.266389,0.593115,-1.135236,-0.163500,-0.263546,0.509675,-0.522625,...,-0.509042,0.844611,-0.584671,-0.261119,3.262728,2.223186,0.619605,0.678672,-0.437484,0.437488
10116,1,-0.664422,1.060261,-0.266389,0.593115,2.180853,-0.163500,-0.263546,-1.368023,0.120645,...,-0.478888,1.255686,-0.591383,0.206786,1.284945,0.176316,-0.121955,0.856430,2.286428,-2.286431
10117,1,-0.290051,-0.943070,-1.036415,-0.893897,0.522808,-0.850632,-0.263546,0.008956,0.763915,...,-0.354313,-1.426735,-0.226314,0.276276,1.734897,-0.207472,0.453393,-0.997334,2.293573,-2.293577
10118,1,-2.037116,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,0.120645,...,-0.368399,-1.426735,-0.240398,-1.039415,1.176215,-0.122186,0.044257,-0.997334,2.290370,-2.290360


In [201]:
preproc_data.Attrition_Flag.value_counts()

1    8494
0    1626
Name: Attrition_Flag, dtype: int64

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,0.431818,1,3,2.0,2.0,70.000000,1,0.604651,5,1,3,0.340190,3.087008e-01,0.344983,0.499066,0.036260,0.250000,0.455055,0.061308,0.999921
1,0.568182,1,3,4.0,2.0,100.000000,1,0.534884,4,1,0,0.059850,3.972984e-10,0.098765,0.969720,0.078753,0.078125,0.653319,0.000001,0.999991
2,0.318182,0,4,2.0,2.0,26.666667,1,0.488372,3,4,1,0.056676,1.000000e+00,0.022779,0.525234,0.037804,0.078125,0.653319,0.763820,0.999881
3,0.318182,1,3,1.0,2.0,70.000000,1,0.186047,5,1,0,0.099091,3.972984e-10,0.136382,0.813084,0.017501,0.140625,0.700084,0.000001,0.999991
4,0.409091,1,2,4.0,2.0,50.000000,1,0.534884,3,1,2,0.077747,4.954311e-01,0.079783,0.514393,0.033057,0.109375,0.236909,0.312564,0.999951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,0.545455,1,2,4.0,1.0,50.000000,1,0.627907,3,2,3,0.077536,7.353993e-01,0.062076,0.262804,0.855934,0.835938,0.239989,0.464323,0.999821
10116,0.340909,1,2,4.0,3.0,50.000000,1,0.279070,4,2,3,0.085819,8.684942e-01,0.060308,0.300561,0.472062,0.460938,0.191263,0.513569,0.004312
10117,0.409091,0,1,2.0,2.0,26.666667,1,0.534884,5,3,4,0.120042,3.972984e-10,0.156466,0.306169,0.559394,0.390625,0.229068,0.000001,0.001700
10118,0.090909,1,2,4.0,2.0,50.000000,1,0.534884,4,3,3,0.116172,3.972984e-10,0.152756,0.200000,0.450958,0.406250,0.202185,0.000001,0.002876


# decide what kind of model is suitable for your task
(supervised / unsupervised learning, classification / regression)
i chose supervised ml because we can organize train/test sets, with given dataset
but also we can try unsupervised ml and compare the results

# decide how to split data into train/test sets
test_size = 0.25   : best_test_size = 0.14909803921568668
0.8944664031620554 : 0.8926441351888668
0.9616600790513834 : 0.9622266401590457
0.9094861660079051 : 0.9105367793240556
0.8525691699604743 : 0.8482438701126573

In [202]:
best_test_size = None

0.14909803921568668

In [10]:
from sklearn.ensemble import RandomForestClassifier

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2,
                                                    random_state=1)
print(f"shapes: \nx_train: {x_train.shape}, \nx_test: {x_test.shape}, \n"
      f"y_train: {y_train.shape}, \ny_test: {y_test.shape}")

y_pos_idx_array = [idx for idx, val in enumerate(y_test) if val == 1]
y_neg_idx_array = [idx for idx, val in enumerate(y_test) if val == 0]

shapes: 
x_train: (8096, 19), 
x_test: (2024, 19), 
y_train: (8096,), 
y_test: (2024,)


0.14909803921568668

# choose the metrics to apply
accuracy, precision, recall, f1_score, confusion_matrix
according to our task we can understand that our goal is in success prediction of positive class of target feature as well as negative one,
so we have to test our model on sets with positive target feature and negative separately and the accuracy wil be the best choice for us
metrics to apply: f1-score

# Naive Bayes model

In [8]:
gnb_model = GaussianNB()
gnb_model.fit(X=x_train, y=y_train)
y_predict = gnb_model.predict(X=x_test)

print("positive target prediction metrics")
# get_metrics(y_test[y_pos_idx_array], y_predict[y_pos_idx_array], mode="show_metrics")
print("negative target prediction metrics")
# get_metrics(y_test[y_neg_idx_array], y_predict[y_neg_idx_array], mode="show_metrics")
get_metrics(y_test, y_predict, mode="show_metrics")
cross_val_score(gnb_model, x, y, cv=10, scoring="f1").mean()

positive target prediction metrics
negative target prediction metrics
accuracy = 0.8898221343873518
confusion_matrix = 
[[ 223  125]
 [  98 1578]]
precision = 0.6408045977011494
recall = 0.6947040498442367
f1_score = 0.6666666666666666


ValueError: 'f1-score' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

# Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier

clf_model = RandomForestClassifier(n_estimators=100)
clf_model.fit(X=x_train, y=y_train)
y_predict = clf_model.predict(x_test)
print("positive target prediction metrics")
# get_metrics(y_test[y_pos_idx_array], y_predict[y_pos_idx_array], mode="show_metrics")
print("negative target prediction metrics")
# get_metrics(y_test[y_neg_idx_array], y_predict[y_neg_idx_array], mode="show_metrics")
get_metrics(y_test, y_predict, mode="show_metrics")
cross_val_score(clf_model, x, y, cv=10, scoring="f1").mean()

positive target prediction metrics
negative target prediction metrics
accuracy = 0.9624505928853755
confusion_matrix = 
[[ 262   17]
 [  59 1686]]
precision = 0.9390681003584229
recall = 0.8161993769470405
f1_score = 0.8733333333333333


0.823359676335591

In [10]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

# tune Random Forest algorithm parameters

In [23]:
from sklearn.model_selection import GridSearchCV
n_estimators = list(range(90, 140 + 10, 10))
max_features = ["auto", "sqrt"]
max_depth = [6, 8, 10]
bootstrap = [True, False]
param_grid = dict(n_estimators=n_estimators, max_features=max_features,
                  max_depth=max_depth, bootstrap=bootstrap)
grid_search = GridSearchCV(clf_model, param_grid=param_grid,
                           scoring="f1", cv=5)
grid_search.fit(X=x, y=y)
clf_model_best_params = grid_search.best_params_

In [24]:
clf_model_best_params

{'bootstrap': False,
 'max_depth': 10,
 'max_features': 'sqrt',
 'n_estimators': 110}

In [224]:
clf_model = RandomForestClassifier(**clf_model_best_params)
cross_val_score(clf_model, x, y, cv=10, scoring="accuracy").mean()

0.9467391304347824

# Support Vector Machine algorithm

In [15]:
from sklearn import svm
svm_svc_model = svm.SVC(kernel="linear")
# svm_svc_model.fit(X=x_train, y=y_train)
y_predict = svm_svc_model.predict(X=x_test)
get_metrics(y_test, y_predict, mode="show_metrics")
cross_val_score(svm_svc_model, x, y, cv=5, scoring="f1").mean()

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# K-nearest neighbor algorithm

In [16]:
from sklearn.neighbors import KNeighborsClassifier

neigh_model = KNeighborsClassifier()
neigh_model.fit(X=x_train, y=y_train)
y_predict = neigh_model.predict(X=x_test)
print("positive target prediction metrics")
# get_metrics(y_test[y_pos_idx_array], y_predict[y_pos_idx_array], mode="show_metrics")
print("negative target prediction metrics")
# get_metrics(y_test[y_neg_idx_array], y_predict[y_neg_idx_array], mode="show_metrics")
get_metrics(y_test, y_predict, mode="show_metrics")
cross_val_score(neigh_model, x, y, cv=10, scoring="accuracy").mean()

positive target prediction metrics
negative target prediction metrics
accuracy = 0.9041501976284585
confusion_matrix = 
[[ 168   41]
 [ 153 1662]]
precision = 0.8038277511961722
recall = 0.5233644859813084
f1_score = 0.6339622641509434


0.899308300395257

# tune K-nearest neighbor algorithm parameters

In [18]:
max_k = 30
knn_scores = []
for k in range(1, max_k + 1):
    neigh_model = KNeighborsClassifier(n_neighbors=k)
    knn_scores.append(cross_val_score(neigh_model, X=x, y=y, cv=10, scoring="accuracy").mean())

print(dict(zip(range(1, max_k + 1), knn_scores)))


{1: 0.875, 2: 0.8857707509881422, 3: 0.8962450592885375, 4: 0.8936758893280633, 5: 0.899308300395257, 6: 0.8956521739130435, 7: 0.899802371541502, 8: 0.8945652173913045, 9: 0.8979249011857708, 10: 0.8958498023715415, 11: 0.8990118577075098, 12: 0.8966403162055336, 13: 0.8976284584980239, 14: 0.8951581027667986, 15: 0.8974308300395257, 16: 0.8950592885375495, 17: 0.8953557312252964, 18: 0.8938735177865613, 19: 0.8955533596837946, 20: 0.8942687747035574, 21: 0.8964426877470355, 22: 0.8933794466403164, 23: 0.8958498023715414, 24: 0.8935770750988145, 25: 0.8958498023715414, 26: 0.8934782608695653, 27: 0.8951581027667986, 28: 0.8927865612648223, 29: 0.8948616600790515, 30: 0.8921936758893281}


In [60]:
best_k = max(zip(range(1, max_k + 1), knn_scores), key=lambda x: x[1])[0]
best_k

11

GridSearchCV

In [19]:
from sklearn.model_selection import GridSearchCV
k_range = range(1, max_k + 1)
weight_options = ["uniform", "distance"]
param_grid = dict(n_neighbors=k_range, weights=weight_options)
grid_search = GridSearchCV(estimator=neigh_model, param_grid=param_grid,
                    cv=5, scoring="f1")
grid_search.fit(X=x, y=y)
best_params = grid_search.best_params_


In [20]:
# neigh_model = KNeighborsClassifier(**best_params)
# cross_val_score(neigh_model, x, y, cv=10, scoring="accuracy").mean()
best_params

{'n_neighbors': 4, 'weights': 'distance'}

RandomizedSearchCV

In [94]:
from sklearn.model_selection import RandomizedSearchCV

rand_search = RandomizedSearchCV(estimator=neigh_model, param_distributions=param_grid,
                          n_iter=5, scoring="accuracy", cv=10)

rand_best_params_list = []
for i in range(10):
    rand_search.fit(X=x, y=y)
    rand_best_params_list.append((rand_search.best_score_, rand_search.best_params_))

best_params = max(rand_best_params_list, key=lambda x: x[0])[1]["n_neighbors"]

[(0.8487154150197629, {'weights': 'distance', 'n_neighbors': 17}),
 (0.849802371541502, {'weights': 'distance', 'n_neighbors': 13}),
 (0.849802371541502, {'weights': 'distance', 'n_neighbors': 13}),
 (0.8467391304347827, {'weights': 'uniform', 'n_neighbors': 17}),
 (0.849802371541502, {'weights': 'distance', 'n_neighbors': 13}),
 (0.8484189723320158, {'weights': 'distance', 'n_neighbors': 18}),
 (0.8463438735177865, {'weights': 'uniform', 'n_neighbors': 18}),
 (0.8485177865612649, {'weights': 'distance', 'n_neighbors': 21}),
 (0.8485177865612649, {'weights': 'distance', 'n_neighbors': 21}),
 (0.8457509881422924, {'weights': 'uniform', 'n_neighbors': 26})]

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,1,-0.165261,1.060261,0.503636,-0.893897,0.522808,0.425471,-0.263546,0.384495,0.763915,...,0.447062,-0.473285,0.489410,2.666758,-0.960392,-0.975049,3.892696,-0.776043,-0.437752,0.437762
1,1,0.583481,1.060261,0.503636,0.593115,0.522808,1.308928,-0.263546,0.008956,0.120645,...,-0.573420,-1.426735,-0.445377,8.499347,-0.741457,-1.913197,6.910076,-0.997334,-0.437950,0.437953
2,1,-0.789213,-0.943070,1.273662,-0.893897,0.522808,-0.850632,-0.263546,-0.241404,-0.522625,...,-0.584975,1.661853,-0.733868,2.991048,-0.952436,-1.913197,6.910076,1.759732,-0.437642,0.437652
3,1,-0.789213,1.060261,0.503636,-1.637403,0.522808,0.425471,-0.263546,-1.868743,0.763915,...,-0.430577,-1.426735,-0.302563,6.558239,-1.057042,-1.572052,7.621803,-0.997334,-0.437949,0.437953
4,1,-0.290051,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,-0.522625,...,-0.508271,0.103448,-0.517445,2.856699,-0.976893,-1.742625,0.572725,0.130886,-0.437857,0.437844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,1,0.458691,1.060261,-0.266389,0.593115,-1.135236,-0.163500,-0.263546,0.509675,-0.522625,...,-0.509042,0.844611,-0.584671,-0.261119,3.262728,2.223186,0.619605,0.678672,-0.437484,0.437488
10116,0,-0.664422,1.060261,-0.266389,0.593115,2.180853,-0.163500,-0.263546,-1.368023,0.120645,...,-0.478888,1.255686,-0.591383,0.206786,1.284945,0.176316,-0.121955,0.856430,2.286428,-2.286431
10117,0,-0.290051,-0.943070,-1.036415,-0.893897,0.522808,-0.850632,-0.263546,0.008956,0.763915,...,-0.354313,-1.426735,-0.226314,0.276276,1.734897,-0.207472,0.453393,-0.997334,2.293573,-2.293577
10118,0,-2.037116,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,0.120645,...,-0.368399,-1.426735,-0.240398,-1.039415,1.176215,-0.122186,0.044257,-0.997334,2.290370,-2.290360
